In [ ]:
from typing import TypedDict, Optional
from langchain_aws import ChatBedrock
from langgraph.graph import StateGraph, START, END
import boto3
import json
import requests
import re

# ==== AWS Configuration ====
REGION = "eu-west-2"
TARGET_LAMBDA = "test_function_26May2025"
KNOWLEDGE_BASE_ID = "ZTPJ2E7RTK"
MODEL_ARN = "arn:aws:bedrock:eu-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"

lambda_client = boto3.client("lambda", region_name=REGION)
brt_client = boto3.client("bedrock-agent-runtime", region_name=REGION)
s3 = boto3.client("s3", region_name=REGION)
bucket = "lhasalimited-gen-ai-unstructured-data-store-transform"

# ==== LangGraph-compatible functions (nodes) ====

def extract_conclusion(narrative: str) -> str:
    match = re.search(r"\*\*Conclusion\*\*:(.*?)(?=\\n\\n|\n\n|\*\*|$)", narrative, re.DOTALL)
    if match:
        return match.group(1).strip()
    return narrative.strip()

def lambda_summary_node(state):
    question = state["question"]
    schema_id = state.get("schema_id", "vitic_lhasa")

    response = lambda_client.invoke(
        FunctionName=TARGET_LAMBDA,
        InvocationType='RequestResponse',
        Payload=json.dumps({
            "question": question,
            "schema_id": schema_id
        })
    )
    payload = json.loads(response['Payload'].read())
    full_narrative = payload.get("response", "")

    if "No available summary" in full_narrative or not full_narrative.strip():
        return {
            **state,
            "narrative_response": None,
            "structured_data_available": False
        }
    else:
        conclusion_only = extract_conclusion(full_narrative)
        return {
            **state,
            "narrative_response": conclusion_only,
            "structured_data_available": True
        }

def fetch_metadata(document_id: str) -> dict:
    key = f"pubmed/{document_id}.metadata.json"
    try:
        response = s3.get_object(Bucket=bucket, Key=key)
        content = response["Body"].read().decode("utf-8")
        metadata_full = json.loads(content)
        metadata = metadata_full.get("metadataAttributes", {})
        print(f"[DEBUG] Metadata keys for {document_id}: {list(metadata.keys())}")
        return metadata
    except Exception as e:
        print(f"Metadata fetch failed for {document_id}: {e}")
        return {}

def bedrock_rag_node(state):
    question = state["question"]
    structured = state.get("narrative_response")

    if structured:
        combined_input = f"[Structured Database Response]\n{structured}\n\n[Follow-up Question]\n{question}"
    else:
        combined_input = f"[Follow-up Question]\n{question}\n(Note: No structured data was available.)"

    citations = []
    metadata_snippets = []

    enriched_prompt = combined_input

    # Pre-fetch citation data before invoking the model
    print("\n[CITATIONS RETURNED FROM RAG]:")
    rag_response = brt_client.retrieve_and_generate(
        input={"text": combined_input},
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": KNOWLEDGE_BASE_ID,
                "modelArn": MODEL_ARN
            }
        }
    )

    citations = rag_response.get("citations", [])
    for i, citation in enumerate(citations, 1):
        print(f"\nCitation {i}:")
        for ref in citation.get("retrievedReferences", []):
            uri = ref.get("location", {}).get("s3Location", {}).get("uri", "N/A")
            snippet = ref.get("content", {}).get("text", "")
            print(f"- URI: {uri}")
            if snippet:
                print(f"  Snippet: {snippet[:200]}...")

    for citation in citations:
        for ref in citation.get("retrievedReferences", []):
            doc_filename = ref.get("location", {}).get("s3Location", {}).get("uri", "")
            doc_id = doc_filename.split("/")[-1]
            metadata = fetch_metadata(doc_id)

            if not metadata:
                continue

            snippet_lines = []
            snippet_lines.append("- Citation Metadata:")

            for field, label in [
                ("title", "Title"),
                ("journal", "Journal"),
                ("year", "Year"),
                ("volume", "Volume"),
                ("issue", "Issue"),
                ("pages", "Pages"),
                ("pmid", "PMID"),
                ("doi", "DOI")
            ]:
                value = metadata.get(field)
                if value:
                    snippet_lines.append(f"  {label}: {value}")

            if metadata.get("chemicals"):
                snippet_lines.append(f"  Chemicals: {', '.join(metadata['chemicals'])}")

            all_keywords = []
            if metadata.get("primaryKeywords"):
                all_keywords.extend(metadata["primaryKeywords"])
            if metadata.get("secondaryKeywords"):
                all_keywords.extend(metadata["secondaryKeywords"])
            if all_keywords:
                snippet_lines.append(f"  Keywords: {', '.join(all_keywords)}")

            if len(snippet_lines) > 1:
                metadata_snippets.append("\n".join(snippet_lines))

    if metadata_snippets:
        enriched_prompt += "\n[Additional Metadata Context]\n" + "\n".join(metadata_snippets)

    print(f"\n[ENRICHED QUESTION SENT TO RAG]:\n{enriched_prompt}")

    return {
        **state,
        "final_answer": rag_response["output"]["text"],
        "citations": citations
    }

def reflector_node(state):
    question = state["question"]
    narrative_response = state.get("narrative_response") or "Prompt only based on the question, no narrative response"

    prompt = f"""
    You are an expert agent. Given the following database response and user question, determine whether the answer is sufficient.

    [Question]
    {question}

    [Narrative Response]
    {narrative_response}

    Respond only as:
    Verdict: COMPLETED or INCOMPLETE
    Justification: (brief explanation)
    """

    model = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", region_name=REGION)
    result = model.invoke(prompt)
    message = result.content

    verdict = "COMPLETED" if "COMPLETED" in message.upper() else "INCOMPLETE"
    justification = message.split("Justification:")[-1].strip() if "Justification:" in message else ""
    retry_count = state.get("retry_count", 0) + 1

    print(f"\n--- Reflector Iteration #{retry_count} ---")
    print(f"Verdict: {verdict}")
    print(f"Justification: {justification}\n")

    return {
        **state,
        "verdict": verdict,
        "justification": justification,
        "retry_count": retry_count
    }

def router(state):
    if not state.get("structured_data_available", True):
        print("No structured data — skipping reflector and going to RAG.")
        return "bedrock_rag"

    retry_count = state.get("retry_count", 0)
    if retry_count >= 10:
        print("Max retry count reached — forcing progression to 'bedrock_rag'")
        return "bedrock_rag"

    if state.get("verdict") == "COMPLETED":
        return "bedrock_rag"
    else:
        return "lambda_summary"

def get_mesh_terms(query, api_key):
    url = "http://data.bioontology.org/annotator"
    params = {
        "text": query,
        "ontologies": "MESH",
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    terms = set()
    for item in data:
        term = item["annotations"][0]["text"]
        terms.add(term)
    return list(terms)

def mesh_api_expansion_node(state):
    query = state["question"]
    api_key = "b41bbf42-2525-4536-b769-6b2cd4b6b302"
    mesh_terms = get_mesh_terms(query, api_key)
    enriched = f"{query}. Related concepts: {', '.join(mesh_terms)}" if mesh_terms else query

    print(f"[Ontology-Enriched Question]: {enriched}")
    return {
        **state,
        "question": enriched,
        "mesh_terms": mesh_terms
    }

# ==== Define the graph ====
class AgentState(TypedDict, total=False):
    question: str
    schema_id: Optional[str]
    narrative_response: Optional[str]
    final_answer: Optional[str]
    retry_count: int
    verdict: Optional[str]
    justification: Optional[str]
    mesh_terms: Optional[list]
    citations: Optional[list]
    structured_data_available: Optional[bool]

graph = StateGraph(AgentState)
graph.add_node("lambda_summary", lambda_summary_node)
graph.add_node("bedrock_rag", bedrock_rag_node)
graph.add_node("reflector", reflector_node)
graph.add_node("mesh_api_expansion", mesh_api_expansion_node)
graph.add_conditional_edges("reflector", router)

# Define transitions
graph.set_entry_point("mesh_api_expansion")
graph.add_edge("mesh_api_expansion", "lambda_summary")
graph.add_edge("lambda_summary", "reflector")
graph.set_finish_point("bedrock_rag")

# Compile the graph
app = graph.compile()





In [ ]:
inputs = {
    "question": "Does Dihydrotestosterone cause breasts cancer?",
    "schema_id": "vitic_lhasa"
}

result = app.invoke(inputs)
print("Final RAG response:\n", result["final_answer"])

[Ontology-Enriched Question]: Does Dihydrotestosterone cause breasts cancer?. Related concepts: BREASTS, DIHYDROTESTOSTERONE, CANCER

--- Reflector Iteration #1 ---
Verdict: INCOMPLETE
Justification: The question asks about the relationship between Dihydrotestosterone (DHT) and breast cancer, but there is no information provided to answer this query. To properly respond, we would need details on whether DHT is linked to an increased risk of breast cancer or not, and any relevant scientific evidence supporting or refuting the connection.

No structured data — skipping reflector and going to RAG.

[CITATIONS RETURNED FROM RAG]:

Citation 1:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/19159686.txt
  Snippet: Androgens have important physiological effects in women. Not only are they the precursor hormones for estrogen biosynthesis in the ovaries and extragonadal tissues, but also act directly via androgen ...

Citation 2:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/8593861.pdf
  Snippet: Interest in the relationship of these chemicals to breast cancer arose because, in addition to their persistence and carcino- genicity, certain of these chemicals are able to mimic the activity of rep...
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/8549499.pdf
  Snippet: Some have focused on potential risks for lung cancer, liver can- cer, lymphoma, soft-tissue sarcoma, and leukemia in case-control studies. However, few of these studies had adequate exposure assessmen...
[DEBUG] Metadata keys for 19159686.txt: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'pmid', 'title', 'chemicals', 'doi']
[DEBUG] Metadata keys for 8593861.pdf: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'primaryKeywords', 'pmid', 'title', 'chemicals', 'doi']
[DEBUG] Metadata keys for 8549499.pdf: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'pmid', 'title', 'chemicals', 'doi']

[ENRICHED QUESTION SENT TO RAG]:
[Follow-up Question]
Does Dihydrotestosterone cause breasts cancer?. Related concepts: BREASTS, DIHYDROTESTOSTERONE, CANCER
(Note: No structured data was available.)
[Additional Metadata Context]
- Citation Metadata:
  Title: Androgen-induced human breast cancer cell proliferation is mediated by discrete mechanisms in estrogen receptor-alpha-positive and -negative breast cancer cells.
  Journal: The Journal of steroid biochemistry and molecular biology
  Year: 2009
  Volume: 113
  Issue: 3-5
  Pages: 182-188
  PMID: 19159686
  DOI: 10.1016/j.jsbmb.2008.12.010
  Chemicals: ESR1 protein, human, Oligopeptides, Androgen Antagonists, arginyl-glycyl-glutamic acid, Integrin alphaVbeta3, Receptors, Androgen, Estradiol, arginyl-glycyl-aspartic acid, Fulvestrant, Androgens, Estrogen Antagonists, Estrogen Receptor alpha, Flutamide, RNA, Small Interfering, AR protein, human
  Keywords: Cell Proliferation, Oligopeptides, Androgen Antagonists, Humans, Integrin alphaVbeta3, Breast Neoplasms, Estradiol, Receptors, Androgen, Fulvestrant, Androgens, Estrogen Antagonists, Estrogen Receptor alpha, Flutamide, Cell Line, Tumor, Female, RNA, Small Interfering
- Citation Metadata:
  Title: Environmental organochlorine exposure as a potential etiologic factor in breast cancer.
  Journal: Environmental health perspectives
  Year: 1995
  Volume: 103 Suppl 7
  Issue: Suppl 7
  Pages: 141-145
  PMID: 8593861
  DOI: 10.1289/ehp.95103s7141
  Chemicals: DDT, Insecticides, Polychlorinated Biphenyls
  Keywords: Environmental Exposure, DDT, Humans, Risk Factors, Female, Insecticides, Polychlorinated Biphenyls, Breast Neoplasms, Case-Control Studies
- Citation Metadata:
  Title: Pesticides--how research has succeeded and failed in informing policy: DDT and the link with breast cancer.
  Journal: Environmental health perspectives
  Year: 1995
  Volume: 103 Suppl 6
  Issue: Suppl 6
  Pages: 87-91
  PMID: 8549499
  DOI: 10.1289/ehp.95103s687
  Chemicals: DDT
  Keywords: DDT, Humans, Female, Research, Breast Neoplasms, Health Priorities, Child
Final RAG response:
 Based on the search results, there is some evidence that dihydrotestosterone (DHT), a metabolite of testosterone, may play a role in breast cancer development and proliferation. Specifically:

- DHT was found to stimulate cell proliferation in both estrogen receptor-positive (MCF-7) and estrogen receptor-negative (MDA-MB-231) human breast cancer cell lines in vitro (source 5).
- In MCF-7 cells, the proliferative effect of DHT appeared to involve the estrogen receptor, as it could be blocked by an estrogen receptor inhibitor or by knocking down estrogen receptor expression (source 5).
- In MDA-MB-231 cells, DHT's effect on proliferation involved a novel mechanism related to the integrin αvβ3, as blocking this integrin eliminated DHT's proliferative effect (source 5). However, the evidence from epidemiological studies linking DHT or its precursor testosterone to breast cancer risk in humans is limited. Most of the evidence relates to the organochlorine pesticide DDT and its metabolite DDE, which have been found to have estrogen-mimicking effects. Several case-control studies reported increased breast cancer risk associated with higher levels of DDE (sources 1 and 3).

In [ ]:
inputs = {
    "question": "Does Phenacetin cause DNA damage, oxidative stress, or cell cycle disruption?",
    "schema_id": "vitic_lhasa"
}

result = app.invoke(inputs)
print("Final RAG response:\n", result["final_answer"])

[Ontology-Enriched Question]: Does Phenacetin cause DNA damage, oxidative stress, or cell cycle disruption?. Related concepts: DNA, CELL CYCLE, CELL, PHENACETIN, DNA DAMAGE, DNA DAMAGE, OXIDATIVE

--- Reflector Iteration #1 ---
Verdict: INCOMPLETE
Justification: The response discusses the carcinogenic potential of Phenacetin in rodents but does not directly address whether Phenacetin causes DNA damage, oxidative stress, or cell cycle disruption, which are the specific mechanisms asked about in the question.


--- Reflector Iteration #2 ---
Verdict: INCOMPLETE
Justification: The response discusses the carcinogenic effects of Phenacetin in rodents but does not directly address whether it causes DNA damage, oxidative stress, or cell cycle disruption, which were the specific mechanisms asked about in the question.


--- Reflector Iteration #3 ---
Verdict: INCOMPLETE
Justification: The response provides evidence of Phenacetin's carcinogenic effects in animal studies, but it does not directly address the question of whether Phenacetin causes DNA damage, oxidative stress, or cell cycle disruption, which are the potential mechanisms underlying its carcinogenicity.


--- Reflector Iteration #4 ---
Verdict: INCOMPLETE
Justification: The response discusses the potential carcinogenic effects of Phenacetin in rodent studies, but it does not directly address whether Phenacetin causes DNA damage, oxidative stress, or cell cycle disruption, which were the specific mechanisms mentioned in the original question.


--- Reflector Iteration #5 ---
Verdict: INCOMPLETE
Justification: The response does not directly address whether Phenacetin causes DNA damage, oxidative stress, or cell cycle disruption, which were the key aspects asked about in the original question. The response focuses on evidence of carcinogenicity in rodents but does not provide information on the specific mechanisms mentioned in the question.


--- Reflector Iteration #6 ---
Verdict: INCOMPLETE
Justification: The given response does not provide any information about whether Phenacetin causes DNA damage, oxidative stress, or cell cycle disruption. The question requires a specific answer addressing the effects of Phenacetin on these cellular processes, but no relevant information is provided.

No structured data — skipping reflector and going to RAG.

[CITATIONS RETURNED FROM RAG]:

Citation 1:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Journal of Toxicology and Environmental Health, Part B/Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf
  Snippet: There are no apparent stu- dies showing genetic or other effects in humans.     Phenacetin induced chromosomal alterations and DNA damage in target and non-target tissues, micronuclei in bone-marrow e...

Citation 2:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Journal of Toxicology and Environmental Health, Part B/Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf
  Snippet: Mitotic recombination and mutation were found in fungi, and mutation and DNA damage in bacteria exposed to PUVA. Treatment with PUVA also produces reactive oxy- gen species (including singlet oxygen a...
Metadata fetch failed for Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
Metadata fetch failed for Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

[ENRICHED QUESTION SENT TO RAG]:
[Follow-up Question]
Does Phenacetin cause DNA damage, oxidative stress, or cell cycle disruption?. Related concepts: DNA, CELL CYCLE, CELL, PHENACETIN, DNA DAMAGE, DNA DAMAGE, OXIDATIVE
(Note: No structured data was available.)
Final RAG response:
 Based on the search results, there is evidence that phenacetin can cause DNA damage and cell cycle disruption:

- Phenacetin induced chromosomal alterations and DNA damage in target and non-target tissues in mice and rats (source 5)
- Phenacetin induced micronuclei (a biomarker of chromosomal damage) in bone-marrow erythrocytes in mice and rats (source 5)
- Phenacetin induced DNA damage in the kidney of mice and in the urinary bladder of rats (source 5)
- Phenacetin induced cell proliferation in the urothelium (epithelial lining) of the kidney, bladder, and renal pelvis in rats, suggesting cell cycle disruption (source 5) Regarding oxidative stress, the search results do not directly mention whether phenacetin causes oxidative stress. However, one result states that treatment with PUVA (psoralen plus UVA radiation), which is a different compound, produces reactive oxygen species that may play a role in cytotoxicity (source 4). But there is no clear evidence provided about phenacetin specifically causing oxidative stress.

In [ ]:
inputs = {
    "question": "What is the carcinogenic potential of diethylstilbestrol?",
    "schema_id": "vitic_lhasa"
}

result = app.invoke(inputs)
print("Final RAG response:\n", result["final_answer"])

[Ontology-Enriched Question]: What is the carcinogenic potential of diethylstilbestrol?. Related concepts: DIETHYLSTILBESTROL

--- Reflector Iteration #1 ---
Verdict: INCOMPLETE
Justification: The given response does not provide any information about the carcinogenic potential of diethylstilbestrol, which was requested in the question. A complete answer should explain whether diethylstilbestrol is carcinogenic or not, and potentially elaborate on its mechanisms or risk factors related to cancer.

No structured data — skipping reflector and going to RAG.

[CITATIONS RETURNED FROM RAG]:

Citation 1:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/1445734.txt
  Snippet: Diethylstilboestrol (DES) exerts several toxic effects in experimental animals, by mechanisms which are still unclear. The genotoxicity of the drug has been attributed to a quinone metabolite and is m...
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/31795923.pdf
  Snippet: To permit a more complete comparison between animal and human tumor sites, tumor sites with only limited evidence in humans are included in Table 8 (in italics). For agents such as diethylstilbestrol ...
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Journal of Toxicology and Environmental Health, Part B/Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf
  Snippet: It is likely that two or more of these factors (see below) in combination are responsible for the carci- nogenic effects of diethylstilbestrol, with estrogen receptor-mediated effects and genotoxicity...

Citation 2:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/31795923.pdf
  Snippet: Concordance may also be elevated if less stringent criteria are applied than those used by IARC for determining sufficient evidence of carcinogenicity in animals. For example, in evaluating available ...
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Tetrahedron Letters/Saeed et al. - 2005 - Novel spiro-quinone formation from 3′-hydroxydieth.pdf
  Snippet: Diethylstilbestrol (DES) is a synthetic estrogen, which was used from 1940 to 1971 as a treatment to prevent spontaneous abortions in women. It is known to be a human carcinogen based on sufficient ev...
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Journal of Toxicology and Environmental Health, Part B/Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf
  Snippet: It is likely that two or more of these factors (see below) in combination are responsible for the carci- nogenic effects of diethylstilbestrol, with estrogen receptor-mediated effects and genotoxicity...
[DEBUG] Metadata keys for 1445734.txt: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'pmid', 'title', 'chemicals', 'doi']
[DEBUG] Metadata keys for 31795923.pdf: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'pmid', 'title', 'chemicals', 'doi']
Metadata fetch failed for Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
[DEBUG] Metadata keys for 31795923.pdf: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'pmid', 'title', 'chemicals', 'doi']
Metadata fetch failed for Saeed et al. - 2005 - Novel spiro-quinone formation from 3′-hydroxydieth.pdf: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
Metadata fetch failed for Birkett et al. - 2019 - Overview of biological mechanisms of human carcino.pdf: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

[ENRICHED QUESTION SENT TO RAG]:
[Follow-up Question]
What is the carcinogenic potential of diethylstilbestrol?. Related concepts: DIETHYLSTILBESTROL
(Note: No structured data was available.)
[Additional Metadata Context]
- Citation Metadata:
  Title: Diethylstilboestrol: II, pharmacology, toxicology and carcinogenicity in experimental animals.
  Journal: European journal of cancer (Oxford, England : 1990)
  Year: 1992
  Volume: 29A
  Issue: 1
  Pages: 149-155
  PMID: 1445734
  DOI: 10.1016/0959-8049(93)90597-9
  Chemicals: Diethylstilbestrol
  Keywords: Cricetinae, Prenatal Exposure Delayed Effects, Genes, Rats, Male, Genital Neoplasms, Male, Mice, Inbred Strains, Pregnancy, Abnormalities, Drug-Induced, Animals, Dogs, Genital Neoplasms, Female, Neoplasms, Experimental, Diethylstilbestrol, Female, Mice
- Citation Metadata:
  Title: Concordance between sites of tumor development in humans and in experimental animals for 111 agents that are carcinogenic to humans.
  Journal: Journal of toxicology and environmental health. Part B, Critical reviews
  Year: 2019
  Volume: 22
  Issue: 7-8
  Pages: 203-236
  PMID: 31795923
  DOI: 10.1080/10937404.2019.1642586
  Chemicals: Carcinogens
  Keywords: Carcinogens, Animals, Species Specificity, Humans, Carcinogenesis, Animals, Laboratory, Neoplasms
- Citation Metadata:
  Title: Concordance between sites of tumor development in humans and in experimental animals for 111 agents that are carcinogenic to humans.
  Journal: Journal of toxicology and environmental health. Part B, Critical reviews
  Year: 2019
  Volume: 22
  Issue: 7-8
  Pages: 203-236
  PMID: 31795923
  DOI: 10.1080/10937404.2019.1642586
  Chemicals: Carcinogens
  Keywords: Carcinogens, Animals, Species Specificity, Humans, Carcinogenesis, Animals, Laboratory, Neoplasms
Final RAG response:
 Diethylstilbestrol (DES) is a synthetic estrogen that has been shown to be carcinogenic in both experimental animals and humans. There is sufficient evidence that DES is carcinogenic in animals after either prenatal or postnatal exposure, causing tumors in various organs including the vagina, cervix, uterus, ovary, mammary gland, and testis in mice. In humans, DES exposure in utero has been linked to an increased risk of clear-cell adenocarcinoma of the vagina and cervix in daughters of women who took DES during pregnancy. The carcinogenic potential of DES is thought to involve both its estrogenic effects as well as metabolic activation to reactive metabolites that can damage cellular macromolecules like DNA. DES is metabolized to catechol metabolites that can be further oxidized to reactive quinones capable of forming DNA adducts, which may lead to cancer-initiating mutations. The developmental changes caused by prenatal DES exposure may also create a tissue environment conducive to carcinogenesis through epigenetic mechanisms.